In [409]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [410]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [411]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [412]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [413]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [414]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [415]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [416]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((20242752370607486079255208826904928047539266627479840103227645476543537159340430211487013570950153776171133327237436754592576420858312736311682916941899324208935787235913483156733665677465221907383627149834646292897572287762639631637848288594239794744751065151439635607746331841895527107807653357072069244532364390428179413396032356338732271663471202100979168691658035324451271842202461601252809601067045448786456677202853124531143555297304191672438383933007458866277428317113006999190738949803501351538848569215722306056273362746434146355059419180472318590268173230087171093620561787516950912542882789557635739418601, 1274714649720670927932455176876953949744319613005708471355562103880970130688068539233786223274158105230891632399594852439212621482397407297020451206934937405009488599395964650545491194980339190377526559931336349642721268287865137050037477178596321454601149206159391144633468205841839153897418979328841632021483894369675576569890199433845429232654254205704915402371275794258443925

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [417]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [418]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [419]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

1344937084062364742044498905070746273990905257942519876113161176846076834030503593228202087568242382346779539481463695632736459511583100638998613517055623789810521019854790793121893638422845522832941576183330264680850856855834643621127250640042128767194696155628665600383927374177694736541116120105009794170711775961606526785868563940566382863824610048728834667773691583671864842061002148500824538449782205623506390766646558813560500468866737939670436661291396352172936366728203546965379650751657784368120940325455998845444064847931558322997078115505422701679925039783647569902759562083401597618828315974312049182053

In [420]:
y_hack = pow(x_hack, e, n)
y_hack

10280242655631921322636331378713458307174644404043936242273503196740347736682871481184532671092046739732703801978974981106450962011553366724196224875544772371735503509551752507987606608826294944702152207005841553933740258966990949265449852190502230665455641592530317354344268355293583472580591982791138536162468892938150733717975001632335165081154912203705277028926994345065594974778602690792529870819144592482418349020960675612951528630438376961833165190814570620194489819247294540006442402186869541618524814841995481599352678215642421325783824156569031860891134160180993827155627114923706162155530895954539758102602

# Hash

In [421]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [422]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

6202448396713572265053540685182382831442032014623950356851809933794532011604795478477483855758228266779052151503133231583584613674120458992696323794117195127067811206913047183203800421627797346447242851760040264924662441911119022079659544301263789407503101088669789689726218047313955463962242070178631777441873212027815877622580978214582919953939588380541520345414522872237772685131862645686612154939313529572285634886037990641499831806088976920709825390049667870327293364991708449730509393291505795423935028850060109639368948547912865550326618544340116878462898465538190836235094282579777293704393810448515847413852

# Decryption of hash (sign) with Sender pub key

In [423]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [424]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

2340509926146499719798124187838309

# Hash verification 

In [425]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(int.from_bytes(final_h.digest()) == decrypt_to_hash)
#print(final_h.digest() == decrypt_to_hash)

True


# Get plaintext

In [426]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

In [427]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Generating keypairs
sender_keypair = generate_keypair() #generate sender keypair
receiver_keypair = generate_keypair() #generate receiver keypair
print(sender_keypair, '\n', receiver_keypair)

#Define message x
msg = 'secret message'.encode('utf-8') #secret message
x = int.from_bytes(msg, 'big') #turn bytes to integer

# Encrypt message x with receiver public key -> ciphertext y
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n) 

# Compute hash
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

#Signing 
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n) #signing hash with sender private key

((14431875022797307767922534821503499950556141244112238095216936994660347087947217006014806040980922775659675483518791600747282189199091826683686472032684413517793729705500853337517677956014064620403142778664018750697507727174484253512069899089851753402802182939448680717189420101453749429476068149163234238758938068632835537719505219529308431136761555373799676622376919579933605686753856031835747957260121981465556738864639752437188068956914255006406017202834578373849775974665456968784326237004013796284328947024859549345432233558171473591267997262728760236941727885969019615015605149959493306397142463856031073991323, 9946309898923249842194993779200531369916154422512389677521163489929467404116958738766948688898058198623750619828808486782330932634248736407874515881695983737036815004899463089755726485734321587186548099384146379800971487168864103018083536444551487928062330111613801876830913736120594000026851730298124260226981016828199846094677880869180639029994195524677477421815365913498086064

In [428]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)

n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)

msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
final_h_int = int.from_bytes(final_h.digest(), 'big')  

valid_signature = decrypt_to_hash == final_h_int
print("Signature valid?", valid_signature)

decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

Signature valid? True


'secret message'

# 3DES

In [429]:
from Crypto.Cipher import DES3, AES
from Crypto.Random import get_random_bytes

def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())


3DES key (hex): 2602f8a86bf83dfb5731324c9732a28a5875b36237a4f808
K1: 2602f8a86bf83dfb
K2: 5731324c9732a28a
K3: 5875b36237a4f808


In [430]:
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

In [431]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)

In [432]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        #pt = unpad(cipher.decrypt(ct), DES3.block_size)
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [433]:
decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'


# AES encryption

In [434]:
from Crypto.Cipher import AES
from Crypto.Util import Counter

nonce = get_random_bytes(8)    # 8-byte nonce (64 bits)
key = get_random_bytes(32)     # AES-256 key

ctr_encrypt = Counter.new(64, prefix=nonce)   # 64-bit counter, prefixed with nonce
aes_cipher_encrypt = AES.new(key, AES.MODE_CTR, counter=ctr_encrypt)
ciphertext = aes_cipher_encrypt.encrypt(data)

encoded_ciphertext = b64encode(ciphertext).decode("utf-8")
print("Ciphertext (Base64):", encoded_ciphertext)



Ciphertext (Base64): bge4CqcmH7eTk8o2EW60IiMn2v8vT7ca8PMrYdnAURE4ne9ETCyYz93D8FSGaVyHB/OXOKARQRXKrnzhzSw4g2hR6WwsdFeYRdL6SdzEaH6t2qi5Qgi+OSJL1J4kBt3XRdov+WYAvnYJrcLhFPhMFZ32pMFwg1Z5WV9U5r0aX8OAzkD85skUUKzAtDu5JZ4AI6EOZJPnqpIvA/n2xQUs51ED611EXsi3vaeqqwJhSWmNSSN80j2OkQaMvKgvmRElfOv1Y6bbg7lF/7uuqGV4NsSgOl2GkETNxDmN9SVyHng6htbpG9SoEYlipARxV4IfsObxMypQHGrJIGMnJRCLyEvDeQBhPoi+8FaEvdeN+cDRappgVFJX+Wxlu2auuqFSvUggKxdDhi+FEymWfjzfJWHFpuFdqrLDR1mOoK8SWVO8wATGWQ5rtitpvgk43Q8GzVkct4azYWXF7m/USYLXaomBrwelX0NeaD4ptNrBFhhXj2l44eOyYRUb110FbJPdySZAmeJTniJWSwxgdgIrUN+cPMAQKnSsdm5RvcVAj5Y0/C3NLVIj63gCtq5rT9+/losfpcM7JPFhYmDY


## AES Decryption

In [435]:

ctr_dec = Counter.new(64, prefix=nonce)
aes_decipher = AES.new(key, AES.MODE_CTR, counter=ctr_dec)
decrypted_bytes = aes_decipher.decrypt(b64decode(encoded_ciphertext))

decrypted_text = decrypted_bytes.decode('utf-8')
print("Decrypted (readable):", decrypted_text)


Decrypted (readable): Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.


## Lesson 7 Python Qs

1. Why must you follow the algorithm steps exactly as prescribed? What happens if you skip some step or apply it in different order?
- ????
2. How did you securely obtain the keys for encryption? How would you exchange these keys to the receiver of you message? How long should the key be considered valid (expiration)? How can you estimate/decide when a key is no longer secure enough?
- For DES, we check if k1 not equal to k2 and k3 
- For AES, it is just a random 256 bit key
- We would exchange using public key encryption like RSA
- 1 message session??? like every reply?? verify with source
- Compute moore's law yaaa research lg

3. Why do we use tripple DES and not a double DES? Is double DES stronger than single DES? Would quadrouple DES be safer to use than tripple DES? Explain.
- if double DES, the key length is only 80 bits effective. yes? because key is longer but idk about computer power HRS SEARCH

4. Why is it advised to have a plain text longer than block size? You encrypted a lorem impsum, which is a clear text with a pattern. Why does your encrypted cipher text has no visible patterns in it? Compare what extra steps in AES are taken to ensure that the plain text pattern is concealed (vs. simple substitution cipher).

BACA LG ANJGGG

5. Can encryption of two different texts result in the same cipher text? How this can be done and what implications does it have on security?

NO?? bcs of iv?? or because of ctr block???? GATAUU WOI


